# Import the libraries

In [5]:
from rdflib import Graph, Literal, term, URIRef, RDF
from rdflib.namespace import Namespace, NamespaceManager
from pyshacl import validate
import re
import os
from rdflib.namespace import Namespace, NamespaceManager

# Parse the graph

# Brick Example

In [ ]:
# Load the Brick graph
brick_graph = Graph()
brick_graph.parse("InstanceModel/brick_model.ttl", format="turtle")

## Parse the graph in memory

In [11]:
#file path to the instance model 
InstanceModel=Graph()
fpath = r'InstanceModel/duplex.ttl'
InstanceModel.parse(fpath,format="turtle")

<Graph identifier=N5b2329b5b3a3455292f7d3935f7cbcde (<class 'rdflib.graph.Graph'>)>

## Bind the name spaces

In [12]:
def bindingNameSpaces(g):
    namespace_manager=NamespaceManager(Graph())
    #User defined namespaces
    INST = Namespace("http://consolidproject.org/models/duplex/" )

    #Namespaces for public available ontologies
    #Unstable namespaces
    BRICK=Namespace("https://brickschema.org/schema/Brick#")
    TAG=Namespace("https://brickschema.org/schema/BrickTag#")
    
    #Stable namespaces
    BOT = Namespace("https://w3id.org/bot#")
    FSO = Namespace("https://w3id.org/fso#")
    QUDT = Namespace("http://qudt.org/schema/qudt/")
    SOSA = Namespace("http://www.w3.org/ns/sosa/")
    QuantityKind = Namespace("http://qudt.org/vocab/quantitykind/")
    RDF = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
    RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
    SSN = Namespace("http://www.w3.org/ns/ssn/")
    GEO = Namespace("http://www.w3.org/2003/01/geo/wgs84_pos#")
    UNIT = Namespace("http://qudt.org/vocab/unit/")
    PROPS = Namespace("https://w3id.org/props#")

    namespace_manager.bind("fso",FSO)
    namespace_manager.bind("sosa",SOSA)
    namespace_manager.bind("ssn",SSN)
    namespace_manager.bind("rdf",RDF)
    namespace_manager.bind("rdfs",RDFS)
    namespace_manager.bind("bot",BOT)
    namespace_manager.bind("tag",TAG)
    namespace_manager.bind("qudt",QUDT)
    namespace_manager.bind("quantitykind",QuantityKind)
    namespace_manager.bind("inst",INST)

In [13]:
#Bind the namespace
bindingNameSpaces(InstanceModel)

# Run a Sparql query

In [ ]:
#A simple query as an example
g = InstanceModel
qres=g.query(
'''
SELECT DISTINCT ?ele ?id
WHERE
{
	?ele a bot:Element;
		 props:batid ?id.
} 
''')
for eleURI, eleID in qres:
	ele = g.namespace_manager.compute_qname(eleURI)[2]
	idBot = g.namespace_manager.compute_qname(eleID)[2]
	print('{0} has ID {1}'.format(ele,idBot))


# Run SCHACL validation

In [16]:
shapes_file = r'Shape/1_Shape.ttl'
shapes_file_format = 'turtle'
data_file_format = 'turtle'
data_file = InstanceModel

conforms, v_graph, v_text = validate(data_file, shacl_graph = shapes_file,
                                     data_graph_format = data_file_format,
                                     shacl_graph_format = shapes_file_format,
                                     inference='rdfs', debug=True,
                                     serialize_report_graph=True)



Cloning DataGraph to temporary memory graph before pre-inferencing.
Running pre-inferencing with option='rdfs'.
Found 1 SHACL Shapes defined with type sh:NodeShape.
Found 0 SHACL Shapes defined with type sh:PropertyShape.
Found 0 property paths to follow.
Found 1 implied SHACL Shapes based on their properties.
Found 1 implied SHACL Shapes used as values in shape-expecting constraints.
Cached 1 unique NodeShapes and 1 unique PropertyShapes.
Will run validation on 1 named graph/s.
Validating DataGraph named N3607bd5deddd443c884c3f6a947faf79
Checking if Shape <NodeShape http://schema.org/SpaceShape> defines its own targets.
Identifying targets to find focus nodes.
Milliseconds to find focus nodes: 0.482ms
Found 21 Focus Nodes to evaluate.
Running evaluation of Shape <NodeShape http://schema.org/SpaceShape>
Current shape evaluation path: <NodeShape http://schema.org/SpaceShape>
Checking conformance for constraint: <PropertyConstraintComponent on <NodeShape http://schema.org/SpaceShape>>
Cu

In [17]:
print('****************************{0}******************************************'.format("Conforms"))
print(conforms)
print('****************************{0}******************************************'.format("V_graph"))
print(v_graph)
print('****************************{0}******************************************'.format("V_text"))
print(v_text)

****************************Conforms******************************************
False
****************************V_graph******************************************
b'@prefix bot: <https://w3id.org/bot#> .\n@prefix inst: <http://consolidproject.org/models/duplex/> .\n@prefix sh: <http://www.w3.org/ns/shacl#> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n[] a sh:ValidationReport ;\n    sh:conforms false ;\n    sh:result [ a sh:ValidationResult ;\n            sh:focusNode inst:space_aa6e13ea-b5bc-4245-93bf-8b23558433df ;\n            sh:resultMessage "Less than 1 values on inst:space_aa6e13ea-b5bc-4245-93bf-8b23558433df->bot:containsElement" ;\n            sh:resultPath bot:containsElement ;\n            sh:resultSeverity sh:Violation ;\n            sh:sourceConstraintComponent sh:MinCountConstraintComponent ;\n            sh:sourceShape _:n175be8fd5d47429face3dbbc4d9eb63ab1 ],\n        [ a sh:ValidationResult ;\n            sh:focusNode inst:space_0b74b3fa-1a92-405e-9ac9-d59067

In [18]:
fpath = fpath.replace('.ttl','_SHACL_validation_report.ttl')
with open(fpath, 'wb') as f:
    f.write(v_graph)